In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import math
%matplotlib inline
plt.style.use('seaborn')
import os

In [3]:
BASE_DIR = '/content/drive/My Drive/4th Year/CS 197 (Thesis)/FINAL - COVID Modeling/notebooks/data/'
file_path = os.path.join(BASE_DIR, 'new_clustered_countries_kshape.csv')
df = pd.read_csv(file_path, index_col=0)

In [4]:
df.head()

,Province/State,Country/Region,Lat,Long,populations,area,1/22/20_confirmed,1/23/20_confirmed,1/24/20_confirmed,1/25/20_confirmed,1/26/20_confirmed,1/27/20_confirmed,1/28/20_confirmed,1/29/20_confirmed,1/30/20_confirmed,1/31/20_confirmed,2/1/20_confirmed,2/2/20_confirmed,2/3/20_confirmed,2/4/20_confirmed,2/5/20_confirmed,2/6/20_confirmed,2/7/20_confirmed,2/8/20_confirmed,2/9/20_confirmed,2/10/20_confirmed,2/11/20_confirmed,2/12/20_confirmed,2/13/20_confirmed,2/14/20_confirmed,2/15/20_confirmed,2/16/20_confirmed,2/17/20_confirmed,2/18/20_confirmed,2/19/20_confirmed,2/20/20_confirmed,2/21/20_confirmed,2/22/20_confirmed,2/23/20_confirmed,2/24/20_confirmed,...,4/17/21_deaths,4/18/21_deaths,shifts,shift 1,shift 2,shift 3,shift 4,shift 5,shift 6,shift 7,shift 8,shift 9,shift 10,shift 11,shift 12,shift 13,shift 14,shift 15,shift 16,shift 17,shift 18,shift 19,shift 20,shift 21,shift 22,shift 23,shift 24,shift 25,shift 26,shift 27,shift 28,shift 29,shift 30,shift 31,shift 32,shift 33,shift 34,shift 35,shift 36,labels
0,no province,Afghanistan,33.93911,67.709953,38928346,no province Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.781541e-10,...,0.003578,0.003644,33,0,0,0,0,0,0,0,1,5,5,5,5,2,0,0,0,0,0,0,0,0,1,2,1,2,1,2,0,0,0,0,0,-1,-1,-1,-1,4
1,no province,Albania,41.15330,20.168300,2877797,no province Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,...,0.106209,0.104009,33,0,0,0,0,0,1,5,5,2,0,1,5,5,5,5,5,5,5,5,5,7,8,8,8,8,8,8,8,8,8,8,8,-1,-1,-1,-1,0
2,no province,Algeria,28.03390,1.659600,43851044,no province Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,...,0.002961,0.002966,33,0,0,0,0,0,0,0,0,0,0,0,1,5,5,5,5,5,2,0,1,5,5,5,5,5,5,5,5,2,0,0,0,-1,-1,-1,-1,2
3,no province,Argentina,-38.41610,-63.616700,45195774,no province Argentina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,...,0.119493,0.126255,32,0,0,0,0,0,0,0,0,1,5,5,5,7,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,-1,-1,-1,-1,-1,1
4,no province,Armenia,40.06910,45.038200,2963243,no province Armenia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,...,0.237354,0.241448,34,0,0,0,0,1,5,5,5,7,8,8,8,8,8,8,4,5,7,8,8,8,8,8,8,8,8,8,4,5,7,8,8,8,-1,-1,-1,1


In [5]:
file_path = os.path.join(BASE_DIR, 'new_clustered_countries_kshape_6months.csv')
df2 = pd.read_csv(file_path, index_col=0)

In [6]:
df2.head()

,Province/State,Country/Region,Lat,Long,populations,area,11/1/20_confirmed,11/2/20_confirmed,11/3/20_confirmed,11/4/20_confirmed,11/5/20_confirmed,11/6/20_confirmed,11/7/20_confirmed,11/8/20_confirmed,11/9/20_confirmed,11/10/20_confirmed,11/11/20_confirmed,11/12/20_confirmed,11/13/20_confirmed,11/14/20_confirmed,11/15/20_confirmed,11/16/20_confirmed,11/17/20_confirmed,11/18/20_confirmed,11/19/20_confirmed,11/20/20_confirmed,11/21/20_confirmed,11/22/20_confirmed,11/23/20_confirmed,11/24/20_confirmed,11/25/20_confirmed,11/26/20_confirmed,11/27/20_confirmed,11/28/20_confirmed,11/29/20_confirmed,11/30/20_confirmed,12/1/20_confirmed,12/2/20_confirmed,12/3/20_confirmed,12/4/20_confirmed,...,3/28/21_deaths,3/29/21_deaths,3/30/21_deaths,3/31/21_deaths,4/1/21_deaths,4/2/21_deaths,4/3/21_deaths,4/4/21_deaths,4/5/21_deaths,4/6/21_deaths,4/7/21_deaths,4/8/21_deaths,4/9/21_deaths,4/10/21_deaths,4/11/21_deaths,4/12/21_deaths,4/13/21_deaths,4/14/21_deaths,4/15/21_deaths,4/16/21_deaths,4/17/21_deaths,4/18/21_deaths,shifts,shift 1,shift 2,shift 3,shift 4,shift 5,shift 6,shift 7,shift 8,shift 9,shift 10,shift 11,shift 12,shift 13,shift 14,shift 15,shift 16,labels
0,no province,Afghanistan,33.93911,67.709953,38928346,no province Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,5.845237e-07,0.000003,0.000009,0.000020,0.000041,0.000074,0.000123,0.000192,0.000282,0.000392,0.000521,0.000667,0.000825,0.000988,0.001152,0.001314,0.001472,0.001624,0.001771,0.001912,0.002049,0.002183,0.002313,0.002437,0.002555,0.002662,0.002757,0.002838,...,0.001020,0.001031,0.001032,0.001022,0.001007,0.000992,0.000985,0.000997,0.001036,0.001111,0.001230,0.001396,0.001608,0.001859,0.002141,0.002440,0.002738,0.003015,0.003256,0.003446,0.003578,0.003644,12,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,2
1,no province,Albania,41.15330,20.168300,2877797,no province Albania,0.0,0.0,0.0,0.0,0.0,0.0,2.951423e-05,0.000150,0.000455,0.001077,0.002184,0.003987,0.006735,0.010604,0.015683,0.021972,0.029368,0.037661,0.046527,0.055528,0.064289,0.072509,0.079975,0.086579,0.092320,0.097313,0.101777,0.105914,0.109891,0.113821,0.117752,0.121662,0.125465,0.129028,...,0.201504,0.195599,0.189708,0.183781,0.177775,0.171688,0.165583,0.159549,0.153682,0.148036,0.142649,0.137567,0.132802,0.128347,0.124201,0.120365,0.116872,0.113744,0.110964,0.108483,0.106209,0.104009,13,1,7,8,8,8,8,8,8,8,8,8,4,-1,-1,-1,-1,1
2,no province,Algeria,28.03390,1.659600,43851044,no province Algeria,0.0,0.0,0.0,0.0,0.0,0.0,2.245638e-06,0.000011,0.000035,0.000084,0.000171,0.000314,0.000534,0.000846,0.001261,0.001782,0.002404,0.003116,0.003895,0.004710,0.005533,0.006339,0.007109,0.007828,0.008488,0.009091,0.009643,0.010152,0.010625,0.011068,0.011482,0.011871,0.012231,0.012560,...,0.002863,0.002804,0.002755,0.002716,0.002686,0.002666,0.002656,0.002656,0.002664,0.002682,0.002707,0.002740,0.002776,0.002813,0.002850,0.002882,0.002909,0.002930,0.002945,0.002954,0.002961,0.002966,13,0,0,1,2,0,0,0,0,0,0,0,0,-1,-1,-1,-1,2
3,no province,Argentina,-38.41610,-63.616700,45195774,no province Argentina,0.0,0.0,0.0,0.0,0.0,0.0,5.659997e-05,0.000281,0.000840,0.001951,0.003884,0.006963,0.011562,0.017880,0.025949,0.035636,0.046651,0.058544,0.070710,0.082388,0.092997,0.102127,0.109531,0.115115,0.118937,0.121201,0.122262,0.122397,0.121822,0.120699,0.119149,0.117256,0.115079,0.112649,...,0.099535,0.099250,0.099062,0.098896,0.098721,0.098546,0.098400,0.098320,0.098356,0.098552,0.098908,0.099384,0.099940,0.100583,0.101386,0.102475,0.104026,0.106282,0.109498,0.113878,0.119493,0.126255,15,1,7,8,4,5,7,8,8,8,4,5,5,7,8,-1,-1,3
4,no province,Armenia,40.06910,45.038200,2963243,no province Armenia,0.0,0.0,0.0,0.0,0.0,0.0,1.689108e-04,0.000844,0.002530,0.005898,0.011781,0.021175,0.035232,0.054578,0.079312,0.108999,0.142673,0.178835,0.215464,0.250027,0.280507,0.305434,0.323894,0.335555,0.340671,0.340089,0.335238,0.327437,0.317864,0.307531,0.297250,0.287610,0.278957,0.271382,...,0.097361,0.107886,0.119347,0.131346,0.143437,0.155207,0.166256,0.176275,0.185040,0.192455,0.198590,0

In [7]:
df_clusters = df[['area', 'Province/State', 'Country/Region', 'labels']]
df_clusters.head()

,area,Province/State,Country/Region,labels
0,no province Afghanistan,no province,Afghanistan,4
1,no province Albania,no province,Albania,0
2,no province Algeria,no province,Algeria,2
3,no province Argentina,no province,Argentina,1
4,no province Armenia,no province,Armenia,1


In [8]:
df2_clusters = df2[['area', 'labels']]
df2_clusters.head()

,area,labels
0,no province Afghanistan,2
1,no province Albania,1
2,no province Algeria,2
3,no province Argentina,3
4,no province Armenia,2


In [ ]:
combined_df = pd.merge(left=df_clusters, right=df2_clusters, on='area')
combined_df.columns = ['area', 'Province/State', 'Country/Region', 'full_cluster', 'last_6months_cluster']
combined_df.head()

,area,Province/State,Country/Region,full_cluster,last_6months_cluster
0,no province Afghanistan,no province,Afghanistan,4,2
1,no province Albania,no province,Albania,0,1
2,no province Algeria,no province,Algeria,2,2
3,no province Argentina,no province,Argentina,1,3
4,no province Armenia,no province,Armenia,1,2


In [ ]:
combined_df['full_cluster'] += 1
combined_df['last_6months_cluster'] += 1
combined_df.head()

,area,Province/State,Country/Region,full_cluster,last_6months_cluster
0,no province Afghanistan,no province,Afghanistan,5,3
1,no province Albania,no province,Albania,1,2
2,no province Algeria,no province,Algeria,3,3
3,no province Argentina,no province,Argentina,2,4
4,no province Armenia,no province,Armenia,2,3


In [ ]:
area = []
for i in combined_df['Province/State']:
    if i=="no province":
        area.append("")
    else:
        area.append(i+" ")
i = 0
for j in combined_df['Country/Region']:
    area[i] += j
    i+=1
combined_df['new_area'] = area
combined_df.head()

,area,Province/State,Country/Region,full_cluster,last_6months_cluster,new_area
0,no province Afghanistan,no province,Afghanistan,5,3,Afghanistan
1,no province Albania,no province,Albania,1,2,Albania
2,no province Algeria,no province,Algeria,3,3,Algeria
3,no province Argentina,no province,Argentina,2,4,Argentina
4,no province Armenia,no province,Armenia,2,3,Armenia


In [ ]:
def get_graphs(df, cluster_num=0, last_columns=71):
  fig=plt.figure(figsize=(50,20))
  ax=fig.add_axes([0,0,1,1])
  ax.set_xlabel("Dates")
  ax.set_ylabel("Cases")

  df_cluster = df[df['labels'] == cluster_num]
  for idx, area in df_cluster.iterrows():
    x = area.index[6:-last_columns]
    y = [area[i] for i in x]
    
    ax.plot(x,y,label=area['Country/Region'])
    #comment out the line below to remove the labels of the area names.
    ax.legend()

In [ ]:
combined_df['change'] = 'F'+combined_df['full_cluster'].astype(str)+'-P'+combined_df['last_6months_cluster'].astype(str)
combined_df.head()

,area,Province/State,Country/Region,full_cluster,last_6months_cluster,new_area,change
0,no province Afghanistan,no province,Afghanistan,5,3,Afghanistan,F5-P3
1,no province Albania,no province,Albania,1,2,Albania,F1-P2
2,no province Algeria,no province,Algeria,3,3,Algeria,F3-P3
3,no province Argentina,no province,Argentina,2,4,Argentina,F2-P4
4,no province Armenia,no province,Armenia,2,3,Armenia,F2-P3


In [ ]:
final = combined_df[['new_area', 'change']]
final.head()

,new_area,change
0,Afghanistan,F5-P3
1,Albania,F1-P2
2,Algeria,F3-P3
3,Argentina,F2-P4
4,Armenia,F2-P3


In [ ]:
ss=''
for i in final[final['change'].str.contains('P4')]['new_area']:
    ss += i + '\n'
print(ss)

Argentina
Austria
Bahrain
Belgium
Botswana
Brazil
Ontario Canada
Chile
Czechia
Dominican Republic
Ecuador
Estonia
Finland
Guadeloupe France
Martinique France
Reunion France
France
Greece
Guatemala
Honduras
Iraq
Italy
Jamaica
Kazakhstan
Kenya
Kosovo
Kuwait
Latvia
Lebanon
Libya
Lithuania
Luxembourg
Malaysia
Mexico
Moldova
Montenegro
Netherlands
Norway
Oman
Paraguay
Peru
Qatar
Serbia
Slovakia
Slovenia
Sweden
US
United Arab Emirates
Uruguay



In [ ]:
final.to_csv(os.path.join(BASE_DIR, 'changes_full_to_partial_kshape.csv'))

In [ ]:
pd.Series(combined_df['change']).value_counts()

03    21
02    20
13    18
42    15
01    10
23     9
40     9
12     9
11     8
32     4
22     4
00     3
10     2
21     2
31     2
20     2
33     1
30     1
Name: change, dtype: int64

## Countries in each change

In [ ]:
combined_df.loc[combined_df['change'] == '03']['area']

6                 no province Austria
11                no province Belgium
16               no province Botswana
25                     Ontario Canada
40     no province Dominican Republic
45                no province Estonia
48                no province Finland
59                 no province Greece
69                  no province Italy
76                 no province Kosovo
79                 no province Latvia
83              no province Lithuania
87               no province Malaysia
89                 no province Mexico
98            no province Netherlands
101                no province Norway
116                no province Serbia
118              no province Slovakia
119              no province Slovenia
124                no province Sweden
134               no province Uruguay
Name: area, dtype: object

In [ ]:
combined_df.loc[combined_df['change'] == '02']['area']

18            no province Bulgaria
22                  Alberta Canada
23         British Columbia Canada
26                   Quebec Canada
27             Saskatchewan Canada
35             no province Croatia
37              no province Cyprus
39             no province Denmark
57             no province Germany
62             no province Hungary
65                no province Iran
72              no province Jordan
100    no province North Macedonia
103           no province Pakistan
109             no province Poland
113             no province Russia
125        no province Switzerland
128             no province Turkey
131            no province Ukraine
135         no province Uzbekistan
Name: area, dtype: object

In [ ]:
combined_df.loc[combined_df['change'] == '13']['area']

3                 no province Argentina
8                   no province Bahrain
17                   no province Brazil
28                    no province Chile
38                  no province Czechia
54                   no province France
60                no province Guatemala
70                  no province Jamaica
73               no province Kazakhstan
80                  no province Lebanon
82                    no province Libya
84               no province Luxembourg
90                  no province Moldova
92               no province Montenegro
102                    no province Oman
106                no province Paraguay
129                      no province US
132    no province United Arab Emirates
Name: area, dtype: object

In [ ]:
combined_df.loc[combined_df['change'] == '42']['area']

0              no province Afghanistan
5                   Victoria Australia
13                   no province Benin
21                no province Cameroon
29                         Hubei China
31     no province Congo (Brazzaville)
34           no province Cote d'Ivoire
42                   no province Egypt
75            no province Korea, South
85              no province Madagascar
99                 no province Nigeria
105       no province Papua New Guinea
123                  no province Sudan
126               no province Thailand
130                 no province Uganda
Name: area, dtype: object

In [ ]:
combined_df.loc[combined_df['change'] == '01']['area']

1           no province Albania
10          no province Belarus
19            no province Burma
46         no province Eswatini
52               Mayotte France
56          no province Georgia
67          no province Ireland
95          no province Namibia
120    no province South Africa
122       no province Sri Lanka
Name: area, dtype: object

In [ ]:
combined_df.loc[combined_df['change'] == '23']['area']

41      no province Ecuador
50        Guadeloupe France
51        Martinique France
53           Reunion France
61     no province Honduras
66         no province Iraq
77       no province Kuwait
107        no province Peru
111       no province Qatar
Name: area, dtype: object

In [ ]:
combined_df.loc[combined_df['change'] == '40']['area']

36                  no province Cuba
44     no province Equatorial Guinea
55                 no province Gabon
58                 no province Ghana
86                no province Malawi
94            no province Mozambique
114               no province Rwanda
138               no province Zambia
139             no province Zimbabwe
Name: area, dtype: object

In [ ]:
combined_df.loc[combined_df['change'] == '12']['area']

4                     no province Armenia
7                  no province Azerbaijan
9                  no province Bangladesh
15     no province Bosnia and Herzegovina
30                   no province Colombia
33                 no province Costa Rica
97                    Curacao Netherlands
112                   no province Romania
137        no province West Bank and Gaza
Name: area, dtype: object

In [ ]:
combined_df.loc[combined_df['change'] == '11']['area']

64          no province Indonesia
68             no province Israel
88              no province Malta
104            no province Panama
110          no province Portugal
121             no province Spain
127           no province Tunisia
133    no province United Kingdom
Name: area, dtype: object